In [ ]:
pip install -e "/Users/adam.santorelli/lib/wristpy/"

In [ ]:
file_name = '/Users/adam.santorelli/Downloads/raw_gt3x_data_archive/NDARPJ763DV1.gt3x'
output_path = '/Users/adam.santorelli/Downloads/raw_gt3x_data_archive/NDARPJ763DV1_output/'


In [70]:
import importlib
import os
import warnings

import polars as pl
from matplotlib import pyplot as plt

import wristpy
from wristpy.common import data_model
from wristpy.ggir import calibration, compare_dataframes, metrics_calc, error_tests
from wristpy.io.loaders import gt3x

warnings.filterwarnings("always")

**TESTING**

In [ ]:
import glob
import os


file_path = "/Users/adam.santorelli/Downloads/raw_gt3x_data_archive/"
ggir_path = "/Users/adam.santorelli/Downloads/ggir_raw_outputs_group1/"

file_name = [os.path.splitext(os.path.basename(file))[0] for file in glob.glob(file_path + "*.gt3x")]
dir_name = [os.path.dirname(file) for file in glob.glob(ggir_path + "*.gt3x")]


In [ ]:
file_name

In [ ]:
importlib.reload(error_tests)
importlib.reload(metrics_calc)

In [ ]:
for file, dir in zip(file_name, dir_name):
    input_path = dir + "/" + file + ".gt3x"
    output_path = dir + "/" + file + "_output/"
    wristpy_out = error_tests.process_file(input_path, output_path)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(wristpy_out.battery_upsample_epoch1)

In [71]:
from wristpy.ggir import error_tests_local

In [ ]:
import glob
import os

file_path = "/Users/adam.santorelli/Downloads/wristpy_out/"
ggir_path = "/Users/adam.santorelli/Downloads/ggir_raw_outputs/"

file_name = [os.path.splitext(os.path.basename(file))[0] for file in glob.glob(file_path + "**/*_metrics_out.csv", recursive=True)]
dir_name =  [os.path.dirname(file) for file in glob.glob(file_path + "**/*_metrics_out.csv", recursive=True)]
#ggir_file_name = [os.path.splitext(os.path.basename(file))[0] for file in glob.glob(ggir_path + "**/*.csv", recursive=True)]

file_name_sorted = sorted(file_name)
dir_name_sorted = sorted(dir_name)


In [ ]:
importlib.reload(error_tests)
importlib.reload(metrics_calc)
importlib.reload(error_tests_local)
importlib.reload(compare_dataframes)

In [ ]:
file_name_sorted[0], dir_name_sorted[0]


In [72]:
errors_out = []

for file, dir in zip(file_name_sorted, dir_name_sorted):
    input_path = dir + "/" + file + ".csv"
    ggir_file_name = os.path.basename(dir).rstrip("_output") 
    ggir_file = ggir_path + ggir_file_name + ".gt3x.RData.csv"
    
    if os.path.exists(ggir_file):
        wristpy_out = pl.read_csv(input_path)
        ggir_data = compare_dataframes.load_ggir_output(ggir_file)
        print(input_path)
        # Compute errors and append to errors_out
        errors = error_tests_local.compute_error(wristpy_out, ggir_data)
        errors_out.append(errors)



/Users/adam.santorelli/Downloads/wristpy_out/NDARAA408JMA_output/NDARAA408JMA_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAA773LUW_output/NDARAA773LUW_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAA948VFH_output/NDARAA948VFH_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAB055BPR_output/NDARAB055BPR_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAB348EWR_output/NDARAB348EWR_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAB653ZXP_output/NDARAB653ZXP_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAB696MF4_output/NDARAB696MF4_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAB708LM5_output/NDARAB708LM5_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAB756JDJ_output/NDARAB756JDJ_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAC350BZ0_output/NDARAC350BZ0_metrics_out.csv
/Users/adam.santorelli/Downloads/wristpy_out/NDARAC350XUM_ou

ComputeError: could not parse `20.0632` as dtype `i64` at column 'anglez' (column number 2)

The current offset in the file is 17967410 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `20.0632` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [ ]:
ggir_data = compare_dataframes.load_ggir_output(ggir_file)


In [ ]:
errors_out

In [ ]:
epoch1_data = compare_dataframes.compare_csv(
        ggir_dataframe=ggir_data, wristpy_dataframe=wristpy_out
    )

In [ ]:
epoch1_data = epoch1_data.with_columns(pl.col("time_epoch1").set_sorted())
NW_flag_rolling_mean = epoch1_data.group_by_dynamic(index_column="time_epoch1", every="3h").agg(pl.col("non_wear_flag").mean())
NW_flag_rolling_mean = NW_flag_rolling_mean["non_wear_flag"].map_elements(
        lambda x: 1 if x > 0.25 else 0
    )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epoch1_data["time_epoch1"], epoch1_data["enmo_wristpy"])

In [ ]:
errors_out

In [ ]:
ggir_time = pl.Series(ggir_data["timestamp"])

ggir_datetime = ggir_time.str.to_datetime(time_unit="ns")

epoch1_wristpy = pl.DataFrame({"time_epoch1": wristpy_out.time_epoch1,
                          "enmo_wristpy": wristpy_out.enmo_epoch1,
                          "anglez_wristpy": wristpy_out.anglez_epoch1,
                          "non_wear_flag": wristpy_out.non_wear_flag_epoch1})

epoch1_ggir = pl.DataFrame({"time_epoch1": ggir_datetime,
                            "enmo_ggir": ggir_data["ENMO"],
                            "anglez_ggir": ggir_data["anglez"]})

epoch1_df_time_match = epoch1_wristpy.join(
        epoch1_ggir, left_on="time_epoch1", right_on="time_epoch1", how="inner")

In [ ]:
epoch1_df_time_match

In [ ]:
NW_flag_rolling_mean = epoch1_df_time_match["non_wear_flag"].rolling_mean(
        window_size=2160
    )
NW_flag_rolling_mean = NW_flag_rolling_mean.map_elements(
        lambda x: 1 if x > 0.25 else 0
    )

metrics_calc_nonwear = epoch1_df_time_match.filter(
        epoch1_df_time_match["non_wear_flag"] == 0
    )

In [ ]:
metrics_calc_nonwear

In [ ]:
plt.plot(metrics_calc_nonwear["time_epoch1"], metrics_calc_nonwear["anglez_wristpy"], label="wristpy")
plt.plot(metrics_calc_nonwear["time_epoch1"], metrics_calc_nonwear["anglez_ggir"], label="ggir")
plt.legend()
plt.show()

In [ ]:
plt.plot( ggir_data["ENMO"])

In [ ]:
np.median([row[2] for row in errors_out])

**GENERAL TESTING**

In [ ]:
file_name = '/Users/adam.santorelli/Downloads/NDARXT222XYH.gt3x'
output_path = '/Users/adam.santorelli/Downloads/NDARXT222XYH_output/'

In [ ]:
test_config = wristpy.common.data_model.Config(file_name, output_path)
test_config.path_output

In [ ]:
import os

file_name = os.path.splitext(os.path.basename(test_config.path_input))[0]
file_name


In [ ]:
import os


last_dir_name = os.path.basename(os.path.dirname(test_config.path_output))
last_dir_name = last_dir_name.split("_output")[0]
last_dir_name

In [ ]:
import importlib
importlib.reload(data_model)
importlib.reload(gt3x)

In [ ]:
test_data = gt3x.load_fast(test_config.path_input)


In [ ]:
test_data

In [ ]:
from wristpy.ggir import metrics_calc
from wristpy.ggir import calibration

In [ ]:
importlib.reload(calibration)

In [ ]:
test_output = calibration.start_ggir_calibration(test_data)

In [ ]:
metrics_calc.calc_base_metrics(test_output)
metrics_calc.calc_epoch1_metrics(test_output)


In [ ]:
metrics_calc.calc_epoch1_raw(test_output)

In [ ]:
importlib.reload(metrics_calc)

In [ ]:
metrics_calc.set_nonwear_flag(test_output, 900)

In [ ]:
metrics_calc.calc_epoch1_light(test_data, test_output)
metrics_calc.calc_epoch1_battery(test_data, test_output)
metrics_calc.calc_epoch1_cap_sensor(test_data, test_output)

In [ ]:
test_output.battery_upsample_epoch1

In [ ]:

import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()

ax1.plot(test_output.time_epoch1, test_output.non_wear_flag_epoch1, color='blue')
ax1.set_ylabel('NW flag', color='blue')

ax2 = ax1.twinx()
ax2.plot(test_output.time_epoch1, test_output.capsense_upsample_epoch1, color='red',alpha=0.5)
ax2.set_ylabel('Cap Sensor', color='red')

# Set y-axis limit to 0
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)


plt.show()


In [ ]:
output_data_csv = pl.DataFrame({
    "time": test_output.time_epoch1,
    "X": test_output.accel_epoch1["X_mean"],
    "Y": test_output.accel_epoch1["Y_mean"],
    "Z": test_output.accel_epoch1["Z_mean"],
    "enmo": test_output.enmo_epoch1,
    "anglez": test_output.anglez_epoch1,
    "Non-wear Flag": test_output.non_wear_flag_epoch1,
    "light": test_output.lux_epoch1,
    "battery voltage": test_output.battery_upsample_epoch1,
    "capacitive sensor": test_output.capsense_upsample_epoch1
})

In [ ]:
import os

output_file_path = test_config.path_output + 'metrics_out.csv'

# Check if the directory already exists
if not os.path.exists(test_config.path_output):
    os.mkdir(test_config.path_output)
else:
    print("Directory already exists.")

if os.path.exists(output_file_path):
    # Generate a new filename, this only allows one copy of _new....
    base_filename = os.path.basename(output_file_path)
    filename, extension = os.path.splitext(base_filename)
    new_filename = filename + "_new" + extension
    
    # Write the CSV file with the new filename
    output_data_csv.write_csv(os.path.join(test_config.path_output, new_filename))
else:
    output_data_csv.write_csv(output_file_path)


In [ ]:
output_file_path = test_config.path_output + 'metrics_out.csv'
output_data_csv.write_csv(output_file_path)

**GIR compare**

In [ ]:
from wristpy.ggir import compare_dataframes
ggir_file = "/Users/adam.santorelli/Downloads/raw_gt3x_data_archive/output_NDARPJ763DV1.gt3x/meta/csv/NDARPJ763DV1.gt3x.RData.csv"
ggir_data = compare_dataframes.load_ggir_output(ggir_file)

In [ ]:
importlib.reload(compare_dataframes)

In [ ]:
difference_df, outputdata_trimmed = compare_dataframes.compare(
                                    ggir_dataframe= ggir_data,
                                    wristpy_dataframe=test_output
)

In [ ]:
outputdata_trimmed = outputdata_trimmed.with_columns(
    ggir_data["ENMO"].alias("GGIR_ENMO"),
    ggir_data["anglez"].alias("GGIR_anglez"))

In [ ]:
NW_flag_rolling_mean = outputdata_trimmed["non-wear flag"].rolling_mean(window_size = 2160)
NW_flag_rolling_mean = NW_flag_rolling_mean.map_elements(lambda x: 1 if x > 0.25 else 0)

In [ ]:
metrics_calc_nonwear = outputdata_trimmed.filter(outputdata_trimmed["non-wear flag"] == 0) 


In [ ]:
metrics_calc_nonwear

In [ ]:
def compute_mse(df, col1, col2):
    squared_error = (df[col1] - df[col2]) ** 2
    mse = squared_error.mean()
    return mse



mse_anglez = compute_mse(metrics_calc_nonwear, "anglez", "GGIR_anglez")
mse_enmo = compute_mse(metrics_calc_nonwear, "ENMO", "GGIR_ENMO")

print(f"MSE ENMO: {mse_enmo}", f"MSE Angle Z: {mse_anglez}")



In [ ]:
angz_diff = (metrics_calc_nonwear["anglez"] - metrics_calc_nonwear["GGIR_anglez"])

plt.hist((angz_diff), bins=90)
plt.xlabel('angz_diff')
plt.ylabel('Frequency')
plt.title('Histogram of angz_diff')
plt.yscale('log')
plt.xlim(-100,100)
plt.show()




In [ ]:
angz_diff.min()

In [ ]:
import plotly.graph_objects as go
import plotly.offline as offline

fig = go.Figure()
fig.add_trace(go.Scatter(x=metrics_calc_nonwear["timestamp"], y=metrics_calc_nonwear["anglez"], mode='lines', name='anglez', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=metrics_calc_nonwear["timestamp"], y=metrics_calc_nonwear["GGIR_anglez"], mode='lines', name='GGIR_anglez', line=dict(color='red')))
fig.add_trace(go.Scatter(x=metrics_calc_nonwear["timestamp"], y=angz_diff, mode='lines', name='angz_diff', line=dict(color='green')))
fig.show()

# Save the plot as an HTML file
#offline.plot(fig, filename='comparison_plot.html')


In [ ]:
difference_df_days = compare_dataframes.select_days(df = difference_df,
                                                       start_day = 2, 
                                                       end_day = 20)
ggir_data_days = compare_dataframes.select_days(df = ggir_data,
                                                   start_day=  2,
                                                   end_day= 20)
outputdata_trimmed_days = compare_dataframes.select_days(df = outputdata_trimmed,
                                                            start_day= 2,
                                                            end_day= 20)

In [ ]:
import numpy as np

(difference_df_days["anglez"] **2).median()

In [ ]:
compare_dataframes.plot_diff(
                outputdata_trimmed = outputdata_trimmed_days, 
                ggir_dataframe = ggir_data_days, 
                difference_df = difference_df_days,
                measures="anglez", 
                opacity= 0.5)

In [ ]:
importlib.reload(compare_dataframes)

**FIND DAYS**


In [ ]:
wristpy_out = pl.read_csv(input_path)
wristpy_time = pl.Series(wristpy_out["time"])
wristpy_datetime = wristpy_time.str.to_datetime(time_unit="ns")
wristpy_out = wristpy_out.replace("time", wristpy_datetime)
import datetime as dt
ordinal_dates = (wristpy_out["time"].dt.timestamp() / (24 * 60 * 60*1e6) + 719163).floor()

In [ ]:
wristpy_out = wristpy_out.with_columns(ordinal_dates.alias("ordinal_dates"))

In [ ]:
wristpy_out

**IAT dates to ordinal**

In [ ]:

IAT_PCIAT_dates = pl.read_csv("/Users/adam.santorelli/Downloads/HBN_IAT_Kaggle2024/IAT_PCIAT_Dates/IAT_PCIAT_dates_sorted_AS.csv")
IAT_dates = IAT_PCIAT_dates["IAT_Date"].str.to_datetime(format="%m/%d/%y")
PCIAT_dates = IAT_PCIAT_dates["PCIAT_Date"].str.to_datetime(format="%m/%d/%y")



In [ ]:
IAT_PCIAT_dates_comp = IAT_PCIAT_dates.with_columns(IAT_dates.alias("IAT_Date"), PCIAT_dates.alias("PCIAT_Date"))

In [ ]:
IAT_PCIAT_dates_comp_filter = IAT_PCIAT_dates_comp.filter(
    (IAT_PCIAT_dates_comp["IAT_Date"].is_not_null()) |
    (IAT_PCIAT_dates_comp["PCIAT_Date"].is_not_null())
)


In [ ]:
IAT_PCIAT_dates_comp_filter

In [ ]:
IAT_ordinal =(IAT_PCIAT_dates_comp_filter["IAT_Date"].dt.timestamp() / (24 * 60 * 60*1e6) + 719163).floor()
PCIAT_ordinal =(IAT_PCIAT_dates_comp_filter["PCIAT_Date"].dt.timestamp() / (24 * 60 * 60*1e6) + 719163).floor()

In [ ]:
IAT_dates_df = pl.DataFrame({"ID": IAT_PCIAT_dates_comp_filter["ID"], "IAT_ordinal": IAT_ordinal, "PCIAT_ordinal": PCIAT_ordinal})

In [ ]:
IAT_dates_df.write_csv("/Users/adam.santorelli/Downloads/HBN_IAT_Kaggle2024/IAT_PCIAT_dates_ordinal.csv")

In [ ]:
IAT_dates_df = pl.read_csv("/Users/adam.santorelli/Downloads/HBN_IAT_Kaggle2024/IAT_PCIAT_dates_ordinal.csv")

In [ ]:
IAT_ordinal_count = IAT_dates_df["IAT_ordinal"].count()
PCIAT_ordinal_count = IAT_dates_df["PCIAT_ordinal"].count()


non_null_count = IAT_dates_df.filter((IAT_dates_df["IAT_ordinal"].is_not_null()) & (IAT_dates_df["PCIAT_ordinal"].is_not_null())).count()
non_null_count,IAT_ordinal_count, PCIAT_ordinal_count



**get relative dates for wristpy_out**

In [ ]:
import datetime as dt
import glob
import os

import pandas as pd
import polars as pl

# Read the Excel file
df = pd.read_csv(
    "/Users/adam.santorelli/Downloads/HBN_IAT_Kaggle2024/IAT_PCIAT_dates_ordinal.csv"
)

# Get the IDs
ids = df["ID"].tolist()

# Find all directories that end with "_output"
directories = glob.glob("/Users/adam.santorelli/Downloads/wristpy_out/*_output")

# Filter the directories based on the IDs from the Excel file
matching_directories = [
    d for d in directories if os.path.basename(d).split("_")[0] in ids
]

# matching_directories now contains all directories that match the IDs from the Excel file

# Save the list of matching directories as a new CSV file
pd.DataFrame(matching_directories, columns=["Directory"]).to_csv(
    "/Users/adam.santorelli/Downloads/wristpy_out/matching_directories.csv", index=False
)

In [ ]:
matching_directories_sorted = sorted(matching_directories)


In [ ]:
len(matching_directories_sorted)

In [ ]:
import numpy as np

In [ ]:
idx = matching_directories_sorted.index("/Users/adam.santorelli/Downloads/wristpy_out/NDARMT703JGP_output")
matching_directories_sorted2 = matching_directories_sorted[idx:]


In [ ]:
# Loop over the matching directories
for directory in matching_directories_sorted2:
    # Get the ID from the directory name
    id = os.path.basename(directory).split("_")[0]

    matching_row = df[df["ID"] == id]

    # Read the CSV file
    wristpy_out = pl.read_csv(f"{directory}/{id}_metrics_out.csv")

    # Convert the time column to ordinal dates
    wristpy_time = pl.Series(wristpy_out["time"])
    wristpy_time = pl.Series(wristpy_out["time"])
    wristpy_datetime = wristpy_time.str.to_datetime(time_unit="ns")

    ordinal_dates = (
        wristpy_out["time"].dt.timestamp() / (24 * 60 * 60 * 1e6) + 719163
    ).floor()

    relative_date_IAT = ordinal_dates - matching_row["IAT_ordinal"].to_numpy()
    relative_date_PCIAT = ordinal_dates - matching_row["PCIAT_ordinal"].to_numpy()
    relative_date_IAT_df = pl.DataFrame({"relative_date_IAT": relative_date_IAT})
    relative_date_PCIAT_df = pl.DataFrame({"relative_date_PCIAT": relative_date_PCIAT})
    wristpy_out = wristpy_out.with_columns(relative_date_IAT_df)
    wristpy_out = wristpy_out.with_columns(relative_date_PCIAT_df)

    # Save the CSV file to the new directory
    wristpy_out.write_csv(
        f"/Users/adam.santorelli/Downloads/wristpy_out_full_dates/{id}_metrics_out_full.csv"
    )


In [ ]:
import glob
# get the mean and std distribution for participants

# Define the directory to search for CSV files
dir_path = "/Users/adam.santorelli/Downloads/wristpy_out_full_dates/"

# Search for all .csv files in the directory
file_list = glob.glob(dir_path + "*.csv")

# Initialize lists to store mean and std values
mean_values = []
std_values = []

# Search the directories for CSV files
for file in file_list:
   
    df = pd.read_csv(file)
    print(f"Processing {file}")
    # Grab the columns IAT_dates and PCIAT_dates
    iat_dates = df["relative_date_IAT"]
    pciat_dates = df["relative_date_PCIAT"]

    # Compute the mean and std of the columns
    mean_iat = iat_dates.mean()
    mean_pciat = pciat_dates.mean()
    std_iat = iat_dates.std()
    std_pciat = pciat_dates.std()

    # Append the mean and std values to the lists
    mean_values.append((mean_iat, mean_pciat))
    std_values.append((std_iat, std_pciat))



In [ ]:
import pandas as pd

# Create a DataFrame from the mean_values and std_values lists
date_data_avg = pd.DataFrame({'mean_iat': [mean[0] for mean in mean_values],
                     'mean_pciat': [mean[1] for mean in mean_values],
                     'std_iat': [std[0] for std in std_values],
                     'std_pciat': [std[1] for std in std_values]})

# Save the DataFrame to a CSV file
date_data_avg.to_csv('mean_std_values.csv', index=False)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Histogram(x=date_data_avg["mean_pciat"], nbinsx=100)])
fig.update_layout(
    xaxis_title="Mean PCIAT Dates",
    yaxis_title="Frequency",
    title="Histogram of Mean IAT Dates"
)
fig.write_html("histogram_PCIAT.html")
fig.show()


In [ ]:
import os
import shutil

# Define source and destination directories
src_dir = '/Users/adam.santorelli//Downloads/ggir_raw_outputs_group1'
dst_dir = '/Users/adam.santorelli//Downloads/ggir_raw_outputs_group2'

# Get list of all files in source directory
src_files = os.listdir(src_dir)

# Iterate over each file in source directory and copy to destination directory
for file_name in src_files:
    full_file_name = os.path.join(src_dir, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy2(full_file_name, dst_dir)

**Additional Sensor data**

In [ ]:
import actfast

subject1 = actfast.read_actigraph_gt3x(test_config.path_input) #this could be anypath to a gt3x file

light_df = pl.from_dict(
    {
    "light_time": subject1["timeseries"]["lux"]["datetime"],
    "lux": subject1["timeseries"]["lux"]["lux"]
})

time_tmp = pl.Series(light_df["light_time"])
time_light = pl.from_epoch(time_tmp, time_unit="ns").alias("time")
light_df = light_df.with_columns(time_light)
light_df_mean = metrics_calc.moving_mean_fast(pl.DataFrame(light_df["lux"]), light_df["time"], 5)

battery_df = pl.from_dict(
    {
    "battery_time": subject1["timeseries"]["battery_voltage"]["datetime"],
    "battery": subject1["timeseries"]["battery_voltage"]["battery_voltage"]
    })

cap_sense_df = pl.from_dict(
    {
        "cap_sense_time": subject1["timeseries"]["capsense"]["datetime"],
        "cap_sense_flag": subject1["timeseries"]["capsense"]["capsense"]    
    })